In [1]:
#importation des modules
import requests
from zipfile import ZipFile
import os
import py7zr
import pandas as pd


In [2]:
#importation des données de niveau de vie en csv 

URL_INSEE_ZIP="https://www.insee.fr/fr/statistiques/fichier/7655475/Filosofi2019_carreaux_200m_csv.zip"
intermediaire_zip="donnees.zip"
destination="/home/onyxia/work/projet_python_data_science"
reponse=requests.get(URL_INSEE_ZIP)

if reponse.status_code == 200 :
    print("téléchargement réussi -> décompression")
    with open(intermediaire_zip,"wb") as f:
        f.write(reponse.content)
    with ZipFile(intermediaire_zip,'r') as zip_ref:
        zip_ref.extractall(destination)
    chemin_fichier_7z=os.path.join(destination,"Filosofi2019_carreaux_200m_csv.7z")
    print("decompression zip terminée -> deuxième décompression")
    with py7zr.SevenZipFile(chemin_fichier_7z,mode="r") as z :
        z.extractall(path=destination)
    #suppression des deux fichiers zip:
    os.remove(os.path.join(destination,intermediaire_zip))
    os.remove(chemin_fichier_7z)
else:
    print(f"echec du télechargement code status : {response.status_code}")

téléchargement réussi -> décompression
decompression zip terminée -> deuxième décompression


In [3]:
#suppression des données inutiles
os.remove(os.path.join(destination,"carreaux_200m_mart.csv"))
os.remove(os.path.join(destination,"carreaux_200m_reun.csv"))


In [4]:
#conversion en dataframe pandas
fichier=os.path.join(destination,"carreaux_200m_met.csv")
df_niveau2vie=pd.read_csv(fichier)

/tmp/ipykernel_147006/2751401850.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_niveau2vie=pd.read_csv(fichier)


In [5]:
#filtrage du data frame sur les communes d'Ile de France

# 1. On force la colonne en texte (string) pour manipuler les caractères
#    Cela gère le cas où "9554160095" serait vu comme un nombre entier
df_niveau2vie['lcog_geo'] = df_niveau2vie['lcog_geo'].astype(str)

# 2. On ne garde STRICTEMENT que les 5 premiers caractères
#    Que le code soit "95541" ou "9554160095", on obtiendra "95541"
df_niveau2vie['code_insee'] = df_niveau2vie['lcog_geo'].str.slice(0, 5)

# 3. Maintenant on peut filtrer pour l'Île-de-France (IDF)
#    Départements IDF : 75, 77, 78, 91, 92, 93, 94, 95
prefixes_idf = ('75', '77', '78', '91', '92', '93', '94', '95')

# On garde ceux qui commencent par un département d'IDF
df_idf = df_niveau2vie[df_niveau2vie['code_insee'].str.startswith(prefixes_idf)].copy()

print(f"Nombre de carreaux en Île-de-France : {len(df_idf)}")
print(df_idf[['lcog_geo', 'code_insee']].head())


Nombre de carreaux en Île-de-France : 74018
        lcog_geo code_insee
65625       9158       9158
65626  915809313      91580
65627  915809313      91580
65628       9158       9158
65629  915809198      91580


In [6]:
print((df_idf['code_insee']).head())

65625     9158
65626    91580
65627    91580
65628     9158
65629    91580
Name: code_insee, dtype: object


In [7]:
# 1. URL du fichier officiel "Base officielle des codes postaux"
# Ce fichier est maintenu à jour et contient la colonne "code_commune_insee" (5 chiffres)
url_communes_propres = "https://www.data.gouv.fr/fr/datasets/r/dbe8a621-a9c4-4bc3-9cae-be1699c5ff25"

print("Chargement du référentiel des communes...")

# 2. Chargement avec les bons paramètres
# sep=';' : car c'est un fichier français
# dtype=str : CRUCIAL pour garder le '0' au début (ex: '01004' et pas '1004')
df_ref_communes = pd.read_csv(url_communes_propres, sep=',', dtype=str)

# 3. Aperçu pour vérifier
# La colonne magique est 'code_commune_insee'
print(df_ref_communes[['code_commune_INSEE', 'nom_commune_postal']].head())

# 4. Préparation pour la fusion
# On ne garde que les colonnes utiles et on renomme pour être clair
df_ref_communes = df_ref_communes[['code_commune_INSEE', 'nom_commune_postal']].drop_duplicates()

# 5. Fusion (Merge) avec votre DataFrame IDF filtré
# Supposons que votre dataframe s'appelle 'df_idf' et la colonne 'code_insee'
df_final_nomme = df_idf.merge(
    df_ref_communes,
    left_on='code_insee',
    right_on='code_commune_INSEE',
    how='left'
)

print("\n--- Résultat après fusion ---")
print(df_final_nomme[['code_insee', 'nom_commune_postal']].head())

Chargement du référentiel des communes...
  code_commune_INSEE       nom_commune_postal
0               1001  L ABERGEMENT CLEMENCIAT
1               1002    L ABERGEMENT DE VAREY
2               1004        AMBERIEU EN BUGEY
3               1005      AMBERIEUX EN DOMBES
4               1006                  AMBLEON

--- Résultat après fusion ---
  code_insee nom_commune_postal
0       9158           LASSERRE
1      91580                NaN
2      91580                NaN
3       9158           LASSERRE
4      91580                NaN


In [8]:
# Affiche le pourcentage de NaN pour chaque colonne (ex: 0.25 pour 25%)
proportions = df_final_nomme['nom_commune_postal'].isna().mean()
print(proportions)
print(len(df_final_nomme))

0.0015536761328325543
74018


In [9]:
print(df_final_nomme[["nom_commune_postal","ind_snv","ind"]])
df_final_nomme["moyenne_niveau_de_vie"]=df_final_nomme["ind_snv"]/df_final_nomme["ind"]
print(df_final_nomme[["nom_commune_postal","moyenne_niveau_de_vie"]])
d_final=df_final_nomme[["nom_commune_postal","moyenne_niveau_de_vie"]]


      nom_commune_postal   ind_snv   ind
0               LASSERRE   18931.8   1.0
1                    NaN   75727.2   4.0
2                    NaN   53776.8   3.0
3               LASSERRE   53776.8   3.0
4                    NaN  233032.8  13.0
...                  ...       ...   ...
74013  ST CLAIR SUR EPTE  244651.4   9.0
74014  ST CLAIR SUR EPTE  145178.4   5.0
74015  ST CLAIR SUR EPTE  116142.7   4.0
74016  ST CLAIR SUR EPTE  290356.7  10.0
74017  ST CLAIR SUR EPTE   43553.5   1.5

[74018 rows x 3 columns]
      nom_commune_postal  moyenne_niveau_de_vie
0               LASSERRE           18931.800000
1                    NaN           18931.800000
2                    NaN           17925.600000
3               LASSERRE           17925.600000
4                    NaN           17925.600000
...                  ...                    ...
74013  ST CLAIR SUR EPTE           27183.488889
74014  ST CLAIR SUR EPTE           29035.680000
74015  ST CLAIR SUR EPTE           29035.675000
74

In [10]:
import unicodedata
d_final = df_final_nomme[["nom_commune_postal","moyenne_niveau_de_vie"]].copy()

# convertir la cible en numérique et supprimer NaN
d_final["moyenne_niveau_de_vie"] = pd.to_numeric(d_final["moyenne_niveau_de_vie"], errors="coerce")
d_final = d_final.dropna(subset=["moyenne_niveau_de_vie", "nom_commune_postal"])

# fonction pour normaliser (minuscules, supprimer accents, trim)
def normalize_name(s):
    if pd.isna(s):
        return s
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = " ".join(s.split())  # collapse spaces
    return s

d_final["nom_norm"] = d_final["nom_commune_postal"].apply(normalize_name)

# Grouper sur nom_norm et calculer la moyenne
result_name = (
    d_final
    .groupby("nom_norm", as_index=False)
    .agg(
        moyenne_niveau_de_vie = ("moyenne_niveau_de_vie", "mean"),
        n_obs = ("moyenne_niveau_de_vie", "count"),
        nom_commune_postal = ("nom_commune_postal", lambda x: x.mode().iat[0] if len(x.mode())>0 else x.iloc[0])
    )
)

result_name["moyenne_niveau_de_vie"] = result_name["moyenne_niveau_de_vie"].round(2)

print(result_name.head())
print(len(result_name))

               nom_norm  moyenne_niveau_de_vie  n_obs    nom_commune_postal
0  abbeville la riviere               26652.90     28  ABBEVILLE LA RIVIERE
1              ableiges               29112.47     32              ABLEIGES
2                 ablis               27816.23     70                 ABLIS
3       ablon sur seine               26161.25     28       ABLON SUR SEINE
4               acheres               27361.23     86               ACHERES
1317


In [11]:
liste_commune=list(result_name["nom_commune_postal"])
result_name.to_csv("donnees_niveau_de_vie.csv",index=False)

In [12]:
#deuxième approche plus compliquée : par les carreaux de 200m directement
# On suppose que votre dataframe s'appelle df_niveau2vie

# 1. Extraction via une expression régulière (Regex)
# On cherche ce qu'il y a entre 'N' et 'E' (le Y), et ce qu'il y a après 'E' (le X)
coord = df_niveau2vie['idcar_200m'].str.extract(r'N(?P<y>\d+)E(?P<x>\d+)')

# 2. Conversion en nombres (car .extract renvoie du texte)
df_niveau2vie['x_crs3035'] = pd.to_numeric(coord['x'])
df_niveau2vie['y_crs3035'] =     pd.to_numeric(coord['y'])

# 3. (Optionnel) Obtenir le CENTRE du carreau
# Comme les coordonnées sont celles du coin bas-gauche et que le carreau fait 200m,
# le centre se trouve à +100m en X et +100m en Y.
df_niveau2vie['x_centre'] = df_niveau2vie['x_crs3035'] + 100
df_niveau2vie['y_centre'] = df_niveau2vie['y_crs3035'] + 100

# Affichage pour vérifier
print(df_niveau2vie[['idcar_200m', 'x_centre', 'y_centre']].head())

                       idcar_200m  x_centre  y_centre
0  CRS3035RES200mN2029800E4252400   4252500   2029900
1  CRS3035RES200mN2029800E4254200   4254300   2029900
2  CRS3035RES200mN2030000E4254200   4254300   2030100
3  CRS3035RES200mN2030000E4254600   4254700   2030100
4  CRS3035RES200mN2030200E4254200   4254300   2030300
